In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('dataframe.csv')
df.to_csv('submission.csv', index=False, header=True)

df.select_dtypes(object)
df.select_dtypes(np.number)

df = df.map(dict)

df.column = df.column.astype(str)

In [2]:
X = 1
y = 2


# Test

In [ ]:
from sklearn import datasets
from sklearn.datasets import fetch_openml

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

data = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)

# Preprocessing

In [ ]:
log_X = np.log(X)
log1p_X = np.log1p(X)

### Fill Nan

In [ ]:
from sklearn.impute import SimpleImputer
def my_function(df):
    df['columna1'] = SimpleImputer(strategy='mean')
    df['columna2'] = SimpleImputer(strategy='median')
    return df

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(X)

### Encode

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit_transform(X)

# METODO 2
X_dummie = pd.get_dummies(X)

### Scale

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_scal = StandardScaler().fit_transform(X)
X_scal = MinMaxScaler().fit_transform(X)

### Polynomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 3)
poly.fit(X)
X_poly = poly.transform(X)
X_poly

### Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
X_kbest = SelectKBest(k=5).fit_transform(X,y)

# Skew

In [ ]:
from scipy.stats import skew
skew_df = pd.DataFrame(df.select_dtypes(np.number).columns, columns=['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: skew(df[feature]))
skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)
skew_df

# Models

## Regression

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lm = LinearRegression()

from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)

from sklearn.svm import SVR
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
from sklearn.svm import LinearSVR
svm_reg = LinearSVR(epsilon=1.5, random_state=42)

from sklearn.ensemble import RandomForestRegressor
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, random_state=42)

from sklearn.ensemble import AdaBoostRegressor
ada_reg = AdaBoostRegressor(n_estimators=200, random_state=42)

from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)

from xgboost import XGBRegressor
xgb_reg = XGBRegressor(random_state=42)

from catboost import CatBoostRegressor
cat_reg = CatBoostRegressor(verbose=0)

from lightgbm import LGBMRegressor
lgb_reg = LGBMRegressor()

from sklearn.ensemble import VotingRegressor
estimators = [('lr', lm), ('rf', tree_reg), ('svc', svm_reg)]
voting_clf = VotingRegressor(estimators=estimators)

from sklearn.linear_model import BayesianRidge, HuberRegressor, OrthogonalMatchingPursuit

lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

sns.scatterplot(y_test, y_pred)
print(lm.intercept_, lm.coef_)
print('Score:', lm.score(X_test,y_test))

## Classification

In [ ]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression()
log_clf.classes_

from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.classes_

from sklearn.svm import SVC
svm_clf = SVC(kernel='linear', C = float(0.5))
svm_clf =  SVC(kernel = "poly", degree=3, coef0=1, C=5)
from sklearn.svm import LinearSVC
Linear_svc = LinearSVC(C = 1, loss='hinge', random_state=42)

from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)

from sklearn.ensemble import AdaBoostClassifier
estimator = DecisionTreeClassifier(max_depth=1)
ada_clf = AdaBoostClassifier(base_estimator = estimator, n_estimators=200, learning_rate=0.5, random_state=42)

from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(max_depth=2, n_estimators=100, learning_rate=1.0, random_state=42)

from xgboost import XGBRFClassifier
xgb_clas = XGBRFClassifier(random_state=42)

from catboost import CatBoostClassifier
cat_clas = CatBoostClassifier(verbose=0)

from lightgbm import LGBMClassifier
lgb_clas = LGBMClassifier()

from sklearn.ensemble import VotingClassifier
estimators = [('lr', log_clf), ('rf', tree_clf), ('svc', svm_clf)]
voting_clf = VotingClassifier(estimators=estimators, voting='hard')

from sklearn.ensemble import BaggingClassifier
estimator = tree_clf
bag_clf = BaggingClassifier(base_estimator = estimator,
    n_estimators=500, # Cantidad de árboles
    max_samples=100, # Muestras utilizadas en bootsrapping
    bootstrap=True, # Usamos bootsrapping
    max_features = 3 # Features que utiliza en el bootsrapping. Cuanto más bajo, mejor generalizará y menos overfitting
    random_state=42)

log_clf.fit(X, y)
y_pred = log_clf.predict(X)
pred_proba = log_clf.predict_proba(X)
log_clf.score(X, y)



# Deep Learning

In [ ]:
from sklearn.linear_model import Perceptron
per_clf = Perceptron(random_state=1)

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=500,activation='tanh',hidden_layer_sizes = (150, 150, 150),random_state=42)

In [ ]:
model = keras.models.Sequential()
# Capa entrada
model.add(keras.layers.Flatten(input_shape=(28, 28)))
# Hidden layer
model.add(keras.layers.Dense(units = 300,activation='relu'))
model.add(keras.layers.Dense(units = 100,activation='relu'))
# Capa salida
model.add(keras.layers.Dense(units = 10,activation='softmax'))

# Otra manera de declarar la red neuronal
capas = [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(units = 300, activation='relu'),
    keras.layers.Dense(units = 100, activation='relu'),
    keras.layers.Dense(units = 10, activation='softmax')
]
model = keras.models.Sequential(capas)

#Regression
model = keras.models.Sequential([
    keras.layers.Dense(30, activation = 'relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error",
             optimizer = "sgd")

model.compile(
    optimizer = "adam", # "adagrad", "adam"
    loss = "sparse_categorical_crossentropy", # binary_crossentropy
    metrics = ["accuracy"]
)

model.summary()
model.evaluate(X_test, y_test)
model.save("my_keras_model.h5")
model = keras.models.load_model("my_keras_model.h5")
checkpoint_cb = keras.callbacks.ModelCheckpoint("callback_model.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5)

history = model.fit(
    X_train,
    y_train,
    batch_size = 128,
    epochs = 50,
    validation_split = 0.1
)

history.params
history.epoch
history.history
history.history.keys()

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

### Combine Predictions

In [ ]:
models = {
    "catboost": CatBoostRegressor(verbose=0),
    "br": BayesianRidge(),
    "lightgbm": LGBMRegressor(),
    "ridge": Ridge(),
    "omp": OrthogonalMatchingPursuit()
}
final_predictions = (
    0.4 * np.exp(models['catboost'].predict(X)) +
    0.2 * np.exp(models['br'].predict(X)) +
    0.2 * np.exp(models['lightgbm'].predict(X)) +
    0.1 * np.exp(models['ridge'].predict(X)) +
    0.1 * np.exp(models['omp'].predict(X))
)

### Regularization

In [ ]:
from sklearn.linear_model import ElasticNet, Ridge, Lasso
ridgeR = Ridge(alpha = 9)
ridgeR.fit(X_train, y_train)
lassoR = Lasso(alpha=42)
lassoR.fit(X_train, y_train)
elastic_net = ElasticNet(alpha = 1, l1_ratio = 0.5)
elastic_net.fit(X_train, y_train)

# Feature Importance

In [ ]:
features = pd.DataFrame(lm.coef_, X.columns, columns=['coefficient']).sort_values('coefficient', ascending=False)
plt.barh(features.index, features.coefficient);

# Evaluate

### Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error, r2_score
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('MAPE:', np.sqrt(mean_squared_error(y_test, y_pred))/y.mean())
print('R2_Score:', r2_score(y_test, y_pred))

### Classification

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
print(accuracy_score(y_test, y_pred))
c_matrix = confusion_matrix(y_test, y_pred)
multilabel_confusion_matrix(y_test, y_pred)



# Grid Search

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

cv = KFold(n_splits=10, random_state=None, shuffle=True)
scores = cross_val_score (X, y, estimator=knn, cv=cv, scoring='accuracy')

params = {'n_neighbors':range(1, 40)}
clf = GridSearchCV(estimator = knn, param_grid = params, scoring='accuracy', cv=cv)

clf.best_params_
clf.best_estimator_
clf.score(X,y)

# Productivization

In [ ]:
import pickle
filename = 'my_model.pkl'
with open(filename, 'wb') as archivo_salida:
    pickle.dump(lm, archivo_salida)
with open(filename, 'rb') as archivo_entrada:
    modelo_importado = pickle.load(archivo_entrada)

from sklearn.pipeline import Pipeline
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("reglog", LogisticRegression())
])

from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), PCA(n_components=4))
pipe.fit(X)

# UNSUPERVISED

# Feature Reduction

In [ ]:
from sklearn.datasets import make_classification

from sklearn.feature_selection import RFE
rfe = RFE(estimator=svm_clf, n_features_to_select=10, step=1)
rfe.ranking_

In [ ]:
from sklearn.feature_selection import VarianceThreshold
x_data_varth = VarianceThreshold(.9).fit_transform(X)

# PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
data = pca.fit_transform(X)

pca.n_components_
pca.explained_variance_ratio_

# Reduce the image dimensions to  2 so that we can visualize the dataset using a Scatterplot.
plt.style.use('seaborn-whitegrid')
plt.figure(figsize = (10,6))
c_map = plt.cm.get_cmap('jet', 10)
plt.scatter(data[:, 0], data[:, 1], s = 15, cmap = c_map , c = y)
plt.colorbar()
plt.xlabel('PC-1') , plt.ylabel('PC-2')
plt.show();

# Clustering

In [4]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=101)
y_pred = kmeans.fit_predict(X)
kmeans.transform(X)

from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5, random_state=42)
minibatch_kmeans.fit(X)

kmeans.score(X)
kmeans.inertia_
kmeans.labels_

# Model Selection

In [ ]:
!pip install pycaret
from pycaret.regression import setup, compare_models
# from pycaret.classification import setup, compare_models
setup(data=pd.concat([X, y], axis=1), target='target')
compare_models()

# Google Colab

In [ ]:
    import os
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.listdir()